# Deep Learning

In [1]:
from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras import regularizers
import keras
import pandas as pd
import numpy as np
from keras import backend as K
from keras import metrics
from collections import namedtuple
pd.set_option("display.max_rows",35)
%matplotlib inline

Using TensorFlow backend.


In [2]:
kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")

#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels_y.pkl")
#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
#y_test_labels = pd.read_pickle("dataset/kdd_test_2labels_y.pkl")

output_columns_2labels = ['is_Attack','is_Normal']

from sklearn import model_selection as ms
from sklearn import preprocessing as pp

x_input = kdd_train_2labels.drop(output_columns_2labels, axis = 1)
y_output = kdd_train_2labels.loc[:,output_columns_2labels]

ss = pp.StandardScaler()
x_input = ss.fit_transform(x_input)

#le = pp.LabelEncoder()
#y_train = le.fit_transform(y_train_labels).reshape(-1, 1)
#y_test = le.transform(y_test_labels).reshape(-1, 1)

y_train = kdd_train_2labels.loc[:,output_columns_2labels]

x_train, x_valid, y_train, y_valid = ms.train_test_split(x_input, 
                              y_train, 
                              test_size=0.1)
#x_valid, x_test, y_valid, y_test = ms.train_test_split(x_valid, y_valid, test_size = 0.4)

x_test = kdd_test_2labels.drop(output_columns_2labels, axis = 1)
y_test = kdd_test_2labels.loc[:,output_columns_2labels]

x_test = ss.transform(x_test)

x_train = np.hstack((x_train, y_train))
x_valid = np.hstack((x_valid, y_valid))

x_test = np.hstack((x_test, np.random.normal(loc = 0, scale = 0.01, size = y_test.shape)))

In [3]:
input_dim = 124
intermediate_dim = 124
latent_dim = 32
batch_size = 1409
epochs = 5
hidden_layers = 8

class Train:
    def train():
        Train.x = Input(shape=(input_dim,))
        
        hidden_encoder = Train.x
        for i in range(hidden_layers):
            hidden_encoder = Dense(intermediate_dim, activation='relu')(hidden_encoder)

        mean_encoder = Dense(latent_dim, activation=None)(hidden_encoder)

        logvar_encoder = Dense(latent_dim, activation=None)(hidden_encoder)

        def get_distrib(args):

            Train.mean_encoder, Train.logvar_encoder = args

            # Sample epsilon
            epsilon = np.random.normal(loc=0.0, scale=0.05, size = (batch_size, latent_dim))

            # Sample latent variable
            z = mean_encoder + K.exp(logvar_encoder / 2) * epsilon
            return z

        z = Lambda(get_distrib)([mean_encoder, logvar_encoder])

        hidden_decoder = z
        for i in range(hidden_layers):
            hidden_decoder = Dense(intermediate_dim, activation="relu")(hidden_decoder)

        Train.x_ = Dense(input_dim, activation=None)(hidden_decoder)

def get_loss(args):
    x, x_ = args
    xent_loss = metrics.binary_crossentropy(x, x_) #input_dim *
    kl_loss = - 0.5 * K.sum(1 + Train.logvar_encoder - K.square(Train.mean_encoder) - K.exp(Train.logvar_encoder), axis=-1)
    label_loss = K.mean(K.argmax(Train.x[:,-2:], axis = 1) - K.argmax(Train.x_[:,-2:], axis = 1))
    
    ls = xent_loss + kl_loss
    #ls += label_loss
    
    return ls



In [4]:
import itertools
#features_arr = [4, 16, 32, 256, 1024]
#hidden_layers_arr = [2, 6, 10, 100]

features_arr = [4, 16, 32, 122]
hidden_layers_arr = [2, 6, 10]

epoch_arr = [50]

def label_accuracy(y_true, y_pred):
    return K.mean(K.equal(K.argmax(y_true, axis = 1), K.argmax(y_pred, axis = 1)))

score = namedtuple("score", ['epoch', 'no_of_features','hidden_layers','train_score', 'test_score'])
scores = []
predictions = pd.DataFrame()

for e, h, f in itertools.product(epoch_arr, hidden_layers_arr, features_arr):
    
    print(" \n Current Layer Attributes - epochs:{} hidden layers:{} features count:{}".format(e,h,f))
    latent_dim = f
    epochs = e
    hidden_layers = h

    Train.train()

    vae_model = Model(inputs = Train.x, outputs = Train.x_ )
    vae_model.compile(optimizer = "adam", loss = "mean_squared_error", metrics = ['accuracy', label_accuracy] )
    #vae_model.compile(optimizer = "adam", loss = Lambda(get_loss)([Train.x, Train.x_]), metrics = ['accuracy', label_accuracy] )

    train_size = x_train.shape[0] - x_train.shape[0]%batch_size
    valid_size = x_valid.shape[0] - x_valid.shape[0]%batch_size

    vae_model.fit(x = x_train[:train_size,:], y = x_train[:train_size,:], 
                  shuffle=True, epochs=epochs, 
                  batch_size = batch_size, 
                  #validation_data = (x_valid[:valid_size,:], x_valid[:valid_size,:]),
                  validation_data = (x_test, x_test),
                  verbose = 1)
    
    score_train = vae_model.evaluate(x_valid[:valid_size,:], y = x_valid[:valid_size,:],
                               batch_size = batch_size,
                               verbose = 1)
    
    score_test = vae_model.evaluate(x_test, y = x_test,
                           batch_size = batch_size,
                           verbose = 1)
    y_test_pred = vae_model.predict(x_test, batch_size=batch_size)
    
    y_pred = np.argmax(y_test_pred[:,-2:], axis = 1)
    y_test_1d = np.argmax(y_test.values, axis = 1)
    
    #y_pred[y_pred >= y_test_pred[:,-1].mean()] = 1
    #y_pred[y_pred < y_test_pred[:,-1].mean()] = 0
    
    label_acc = np.mean(np.equal(y_test_1d, y_pred))
    
    scores.append(score(e,f,h,score_train[-1], label_acc)) #score_test[-1]))
    
    curr_pred = pd.DataFrame({"{}_{}_{}".format(e,f,h):y_pred},)
    predictions = pd.concat([predictions, curr_pred], axis = 1)
    
    print("\n Train Acc: {}, Test Acc: {}, Label Acc: {}".format(score_train[-1], 
                                                                 score_test[-1], 
                                                                 label_acc)  )
    
scores = pd.DataFrame(scores)

    

 
 Current Layer Attributes - epochs:50 hidden layers:2 features count:4
Train on 112720 samples, validate on 22544 samples
Epoch 1/50
112720/112720 [==============================] - 1s - loss: 1.6581 - acc: 0.0165 - label_accuracy: 0.0165 - val_loss: 1.7722 - val_acc: 0.0841 - val_label_accuracy: 0.0841
Epoch 2/50
112720/112720 [==============================] - 1s - loss: 276.1450 - acc: 0.2064 - label_accuracy: 0.2064 - val_loss: 1.7912 - val_acc: 0.3401 - val_label_accuracy: 0.3401
Epoch 3/50
112720/112720 [==============================] - 0s - loss: 0.7537 - acc: 0.2612 - label_accuracy: 0.2612 - val_loss: 1.6966 - val_acc: 0.3467 - val_label_accuracy: 0.3467
Epoch 4/50
112720/112720 [==============================] - 0s - loss: 0.6940 - acc: 0.5030 - label_accuracy: 0.5030 - val_loss: 1.6625 - val_acc: 0.6281 - val_label_accuracy: 0.6281
Epoch 5/50
112720/112720 [==============================] - 0s - loss: 0.6735 - acc: 0.5327 - label_accuracy: 0.5327 - val_loss: 1.6423 - val_

112720/112720 [==============================] - 1s - loss: 0.3913 - acc: 0.7055 - label_accuracy: 0.7055 - val_loss: 1.0318 - val_acc: 0.8428 - val_label_accuracy: 0.8428
Epoch 37/50
112720/112720 [==============================] - 1s - loss: 0.3896 - acc: 0.7103 - label_accuracy: 0.7103 - val_loss: 1.0252 - val_acc: 0.8341 - val_label_accuracy: 0.8341
Epoch 38/50
112720/112720 [==============================] - 1s - loss: 0.3885 - acc: 0.6997 - label_accuracy: 0.6997 - val_loss: 1.0161 - val_acc: 0.8283 - val_label_accuracy: 0.8283
Epoch 39/50
112720/112720 [==============================] - 1s - loss: 0.3869 - acc: 0.7292 - label_accuracy: 0.7292 - val_loss: 1.0161 - val_acc: 0.8325 - val_label_accuracy: 0.8325
Epoch 40/50
112720/112720 [==============================] - 1s - loss: 0.3855 - acc: 0.7279 - label_accuracy: 0.7279 - val_loss: 1.0216 - val_acc: 0.8439 - val_label_accuracy: 0.8439
Epoch 41/50
112720/112720 [==============================] - 1s - loss: 0.3841 - acc: 0.7332

112720/112720 [==============================] - 1s - loss: 0.9889 - acc: 0.0000e+00 - label_accuracy: 0.0000e+00 - val_loss: 1.9633 - val_acc: 0.0000e+00 - val_label_accuracy: 0.0000e+00
Epoch 18/50
112720/112720 [==============================] - 1s - loss: 0.9889 - acc: 0.0000e+00 - label_accuracy: 0.0000e+00 - val_loss: 1.9633 - val_acc: 0.0000e+00 - val_label_accuracy: 0.0000e+00
Epoch 19/50
112720/112720 [==============================] - 1s - loss: 0.9889 - acc: 0.0000e+00 - label_accuracy: 0.0000e+00 - val_loss: 1.9633 - val_acc: 0.0000e+00 - val_label_accuracy: 0.0000e+00
Epoch 20/50
112720/112720 [==============================] - 1s - loss: 0.9889 - acc: 0.0000e+00 - label_accuracy: 0.0000e+00 - val_loss: 1.9633 - val_acc: 0.0000e+00 - val_label_accuracy: 0.0000e+00
Epoch 21/50
112720/112720 [==============================] - 1s - loss: 0.9889 - acc: 0.0000e+00 - label_accuracy: 0.0000e+00 - val_loss: 1.9633 - val_acc: 0.0000e+00 - val_label_accuracy: 0.0000e+00
Epoch 22/50


Train on 112720 samples, validate on 22544 samples
Epoch 1/50
112720/112720 [==============================] - 4s - loss: 1.2773 - acc: 0.0620 - label_accuracy: 0.0620 - val_loss: 1.8103 - val_acc: 0.2112 - val_label_accuracy: 0.2112
Epoch 2/50
112720/112720 [==============================] - 3s - loss: 0.7548 - acc: 0.3573 - label_accuracy: 0.3573 - val_loss: 1.7016 - val_acc: 0.3506 - val_label_accuracy: 0.3506
Epoch 3/50
112720/112720 [==============================] - 3s - loss: 0.6520 - acc: 0.5683 - label_accuracy: 0.5683 - val_loss: 1.6174 - val_acc: 0.6113 - val_label_accuracy: 0.6113
Epoch 4/50
112720/112720 [==============================] - 3s - loss: 0.5930 - acc: 0.6673 - label_accuracy: 0.6673 - val_loss: 1.5312 - val_acc: 0.7381 - val_label_accuracy: 0.7381
Epoch 5/50
112720/112720 [==============================] - 3s - loss: 0.5350 - acc: 0.7566 - label_accuracy: 0.7566 - val_loss: 1.4787 - val_acc: 0.7889 - val_label_accuracy: 0.7889
Epoch 6/50
112720/112720 [========

112720/112720 [==============================] - 3s - loss: 0.2766 - acc: 0.7697 - label_accuracy: 0.7697 - val_loss: 0.6388 - val_acc: 0.6978 - val_label_accuracy: 0.6978
Epoch 39/50
112720/112720 [==============================] - 3s - loss: 0.2765 - acc: 0.7643 - label_accuracy: 0.7643 - val_loss: 0.6887 - val_acc: 0.7376 - val_label_accuracy: 0.7376
Epoch 40/50
112720/112720 [==============================] - 3s - loss: 0.2740 - acc: 0.8012 - label_accuracy: 0.8012 - val_loss: 0.6400 - val_acc: 0.7349 - val_label_accuracy: 0.7349
Epoch 41/50
112720/112720 [==============================] - 3s - loss: 0.2754 - acc: 0.7981 - label_accuracy: 0.7981 - val_loss: 0.6308 - val_acc: 0.7355 - val_label_accuracy: 0.7355
Epoch 42/50
112720/112720 [==============================] - 3s - loss: 0.2725 - acc: 0.7941 - label_accuracy: 0.7941 - val_loss: 0.6093 - val_acc: 0.7421 - val_label_accuracy: 0.7421
Epoch 43/50
112720/112720 [==============================] - 3s - loss: 0.2604 - acc: 0.7802

112720/112720 [==============================] - 3s - loss: 0.2908 - acc: 0.8610 - label_accuracy: 0.8610 - val_loss: 0.8708 - val_acc: 0.8364 - val_label_accuracy: 0.8364
Epoch 25/50
112720/112720 [==============================] - 3s - loss: 0.3009 - acc: 0.8546 - label_accuracy: 0.8546 - val_loss: 0.8669 - val_acc: 0.8312 - val_label_accuracy: 0.8312
Epoch 26/50
112720/112720 [==============================] - 3s - loss: 0.3487 - acc: 0.8273 - label_accuracy: 0.8273 - val_loss: 1.1238 - val_acc: 0.7378 - val_label_accuracy: 0.7378
Epoch 27/50
112720/112720 [==============================] - 2s - loss: 0.3294 - acc: 0.8132 - label_accuracy: 0.8132 - val_loss: 1.1082 - val_acc: 0.7496 - val_label_accuracy: 0.7496
Epoch 28/50
112720/112720 [==============================] - 2s - loss: 0.3138 - acc: 0.7995 - label_accuracy: 0.7995 - val_loss: 0.9623 - val_acc: 0.7316 - val_label_accuracy: 0.7316
Epoch 29/50
112720/112720 [==============================] - 2s - loss: 0.2721 - acc: 0.8472

112720/112720 [==============================] - 2s - loss: 0.5284 - acc: 0.7209 - label_accuracy: 0.7209 - val_loss: 1.4148 - val_acc: 0.7000 - val_label_accuracy: 0.7000
Epoch 11/50
112720/112720 [==============================] - 2s - loss: 0.5069 - acc: 0.7416 - label_accuracy: 0.7416 - val_loss: 1.3870 - val_acc: 0.7363 - val_label_accuracy: 0.7363
Epoch 12/50
112720/112720 [==============================] - 2s - loss: 0.5035 - acc: 0.7238 - label_accuracy: 0.7238 - val_loss: 1.3652 - val_acc: 0.6976 - val_label_accuracy: 0.6976
Epoch 13/50
112720/112720 [==============================] - 2s - loss: 0.4824 - acc: 0.7427 - label_accuracy: 0.7427 - val_loss: 1.4022 - val_acc: 0.7331 - val_label_accuracy: 0.7331
Epoch 14/50
112720/112720 [==============================] - 2s - loss: 0.4972 - acc: 0.7181 - label_accuracy: 0.7181 - val_loss: 1.3456 - val_acc: 0.6971 - val_label_accuracy: 0.6971
Epoch 15/50
112720/112720 [==============================] - 2s - loss: 0.4963 - acc: 0.7197

112720/112720 [==============================] - 2s - loss: 0.2104 - acc: 0.8905 - label_accuracy: 0.8905 - val_loss: 0.9553 - val_acc: 0.8129 - val_label_accuracy: 0.8129
Epoch 48/50
112720/112720 [==============================] - 2s - loss: 0.2330 - acc: 0.8943 - label_accuracy: 0.8943 - val_loss: 0.9992 - val_acc: 0.7960 - val_label_accuracy: 0.7960
Epoch 49/50
112720/112720 [==============================] - 2s - loss: 0.2216 - acc: 0.8942 - label_accuracy: 0.8942 - val_loss: 0.9202 - val_acc: 0.8104 - val_label_accuracy: 0.8104
Epoch 50/50
18317/22544 [=======================>......] - ETA: 0s
 Train Acc: 0.9054293930530548, Test Acc: 0.816270399838686, Label Acc: 0.7818488289567069


In [5]:
scores.sort_values("test_score", ascending=False)

,epoch,no_of_features,hidden_layers,train_score,test_score
10,50,32,10,0.878726,0.822214
7,50,122,6,0.736515,0.821283
1,50,16,2,0.765703,0.812944
9,50,16,10,0.882630,0.802253
4,50,4,6,0.902147,0.792362
11,50,122,10,0.905429,0.781849
5,50,16,6,0.936125,0.781317
0,50,4,2,0.844571,0.775417
8,50,4,10,0.870653,0.755456
6,50,32,6,0.891945,0.753504


In [6]:
predictions.to_pickle("dataset/vae_only_predictions.pkl")
scores.to_pickle("dataset/vae_only_scores.pkl")